### 그래픽 카드 설정 및 확인
**torch로 그래픽카드가 잡히지 않을 때만 확인용으로 주석 제거 후 돌리기**

토치 버전 확인

In [1]:
%pip show torch

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: c:\users\ssafy\desktop\wang\kcvenv\lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, bitsandbytes, peft, torchaudio, torchvision
Note: you may need to restart the kernel to use updated packages.


CUDA 확인

In [2]:
import torch

print(torch.cuda.get_device_name(0))

#!nvidia-smi

NVIDIA GeForce RTX 4050 Laptop GPU


### 로컬에선 GPU 지정 필요없음. GPU 서버에서는 주석 풀기

In [ ]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

- 현재 파일에서는 hugginface의 beomi/KcELECTRA-base 모델을 사용할 예정입니다.
- 해당 모델에 관한 정보는 config 파일을 통해 관리합니다.
- 다른 모델을 사용하기 원할 경우 config의 MODEL_NAME을 huggingface의 가이드에 따라 변경하면 됩니다.

## 성능 향상을 위한 Text 정제 코드
- huggingface에서 개발자가 공개한 성능을 높이기 위한 데이터 전처리 과정입니다. 특수문자 및 이모지 등을 제거합니다
- 불용어는 제거하지 않습니다. 감정 모델에서는 불용어 또한 중요한 데이터일 수 있습니다.

In [4]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = ''.join(emoji.EMOJI_DATA.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

import re
import emoji
from soynlp.normalizer import repeat_normalize

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x): 
    x = pattern.sub(' ', x)
    x = emoji.replace_emoji(x, replace='') #emoji 삭제
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

### 텍스트 정제 확인

In [5]:
sentence = "나는 오늘 선배한테 혼났어 ★."
cleaned_sentence = clean(sentence)
print(cleaned_sentence)

나는 오늘 선배한테 혼났어 .


In [ ]:
sentiment_label = {
    # 우울함, 무기력력
    "depression": [
        "depressive_mood", "anhedonia", "worthlessness", "negative_self-image"
    ],
    # 불안, 긴장장
    "nervous": [
        "loss_of_control", "emotional_requlation", "coping"
    ],
    # 죄책감·수치심·자책 관련
    "guilt": [
        "guilt", "belief"
    ],
    # 동기·회복 기대 관련
    "motivation": [
        "motivation_for_change", "unrealistic_recovery_expectations", "belief"
    ],
    # 인지 저하·비현실적 사고 관련
    "cognitive_decline": [
        "impaired_cognition", "unrealistic_recovery_expectations"
    ],
    # 신체적 증상 및 행동 변화 관련
    "somatic": [
        "fatigue", "psychomotor_changes", "sleep_disturbance", "weight_appetite", "lifestyle"
    ],
    # 외상·트라우마 관련
    "trauma": [
        "trauma_experience", "loss_of_control", "emotional_requlation"
    ],
    # 환경·가족·기저질환 등 배경 요인
    "background": [
        "family_history", "underlying_physical_condition"
    ],
}

Emotion_Classification = list(sentiment_label.keys())

# json 파일을 까서 세부 감정분류를 대분류로 매핑시키기 위한 코드
def class_target(item: list):
    y = []
    for category, sub_label in Emotion_Classification.items():
        val = 0
        for sub in sub_label:
            if sub in item and item[sub] > 0:
                val = 1
                break
            y.append(val)
    return y

['depression', 'nervous', 'guilt', 'motivation', 'cognitive_decline', 'somatic', 'trauma', 'background']


## 학습 실행

#### 1. 모델 설정

In [ ]:
import os
import json
import random
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.metrics import precision_recall_fscore_support, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from config import ModelConfig as cf

model_name = cf.MODEL_NAME
DATA_DIR = r"C:\Users\SSAFY\Desktop\WANG\S13P31A106\KcELECTRA\data\Aftre_processed"
OUTPUT_DIR = r"C:\Users\SSAFY\Desktop\WANG\S13P31A106\KcELECTRA\checkpoints"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 모델 설정
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(Emotion_Classification),
    problem_type="multi_label_classification"
)

#### 2. 추론 카테고리 설정

In [ ]:
# 감정 카테고리
CATEGORIES = [
    "depression", "nervous", "guilt", "motivation",
    "cognitive_decline", "somatic", "trauma", "background"
]
NUM_LABELS = len(CATEGORIES)

#### 3. 데이터셋 로드

In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, json_files, tokenizer, max_len=256):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        for path in json_files:
            with open(path, "r", encoding="utf-8-sig") as f:
                samples = json.load(f)
            for s in samples:
                if not s.get("text") or not s.get("labels"):
                    continue
                text = s["text"].strip()
                labels = s["labels"]
                # 8개 카테고리 모두 포함 (없는 건 0)
                y = [labels.get(cat, 0) for cat in CATEGORIES]
                self.data.append({"text": text, "labels": y})

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        enc = self.tokenizer(
            item["text"],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(item["labels"], dtype=torch.float)
        }

#### 4.학습 준비

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs >= 0.5).astype(int)

    micro = precision_recall_fscore_support(labels, preds, average="micro", zero_division=0)
    macro = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
    samples_f1 = f1_score(labels, preds, average="samples", zero_division=0)

    return {
        "micro_f1": micro[2],
        "macro_f1": macro[2],
        "samples_f1": samples_f1
    }

tokenizer = AutoTokenizer.from_pretrained(model_name)

# 파일 분할 (세션 단위)
files = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith(".json")]
random.shuffle(files)
split_idx = int(len(files) * 0.8)
train_files, val_files = files[:split_idx], files[split_idx:]

train_ds = EmotionDataset(train_files, tokenizer)
val_ds = EmotionDataset(val_files, tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    report_to="none",
    fp16=torch.cuda.is_available(),
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model(os.path.join(OUTPUT_DIR, "best_model"))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "best_model"))

# 추론

In [ ]:
def predict_emotion(text: str):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.sigmoid(logits).squeeze().tolist()
    return {CATEGORIES[i]: round(float(probs[i]), 3) for i in range(NUM_LABELS)}

print(predict_emotion("요즘 너무 무기력하고 잠도 잘 안 와요."))

## LORA 가중치 병합